# 1. Setup Mario

In [1]:
!pip install gym_super_mario_bros==7.3.0 nes_py

Defaulting to user installation because normal site-packages is not writeable


In [2]:
# Game
import gym_super_mario_bros
# Joypad
from nes_py.wrappers import JoypadSpace
# Controls
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

In [3]:
SIMPLE_MOVEMENT

[['NOOP'],
 ['right'],
 ['right', 'A'],
 ['right', 'B'],
 ['right', 'A', 'B'],
 ['A'],
 ['left']]

In [4]:
# Setup game
env = gym_super_mario_bros.make('SuperMarioBros-v0')
# Simplify movement variations (action_space)
env = JoypadSpace(env, SIMPLE_MOVEMENT)

In [5]:
# Flag
done = True

# Loop through each frame
for step in range(100):
    if done:
        # Start the game
        env.reset()
    # Do random actions
    state, reward, done, info = env.step(env.action_space.sample())
    # Show the game on the screen
    env.render()
env.close()

C:\Users\yacoo\AppData\Roaming\Python\Python310\site-packages\pyglet\image\codecs\wic.py:289: UserWarning: [WinError -2147417850] Cannot change thread mode after it is set
  warnings.warn(str(err))


# 2. Preprocess Environment

In [6]:
!pip install torch torchvision torchaudio
# PPO for RL
!pip install stable-baselines3[extra]

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [7]:
# Frame Stacker Wrapper and Grayscaling Wrapper
# Framestack: Capture last couple of frames while Mario is playing
# GrayscaleObservation: Shave our data since it does not deal with no coloring, making our model faster
from gym.wrappers import GrayScaleObservation
# Vectorization Wrappers
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv
# Plot the impact of framestacking
from matplotlib import pyplot as plt

In [8]:
# Create the base env 
env = gym_super_mario_bros.make('SuperMarioBros-v0')
# Simplify movement controls
env = JoypadSpace(env, SIMPLE_MOVEMENT)
# Grayscale the env
env = GrayScaleObservation(env, keep_dim=True)
# Wrap inside the dummy env
env = DummyVecEnv([lambda: env])
# Stack the frames
# Find how many frames to take in as you desire that works best
env = VecFrameStack(env, 4, channels_order='last')

In [9]:
# state = env.reset()
# plt.imshow(state)

In [10]:
env.reset()

array([[[[  0,   0,   0, 140],
         [  0,   0,   0, 140],
         [  0,   0,   0, 140],
         ...,
         [  0,   0,   0, 140],
         [  0,   0,   0, 140],
         [  0,   0,   0, 140]],

        [[  0,   0,   0, 140],
         [  0,   0,   0, 140],
         [  0,   0,   0, 140],
         ...,
         [  0,   0,   0, 140],
         [  0,   0,   0, 140],
         [  0,   0,   0, 140]],

        [[  0,   0,   0, 140],
         [  0,   0,   0, 140],
         [  0,   0,   0, 140],
         ...,
         [  0,   0,   0, 140],
         [  0,   0,   0, 140],
         [  0,   0,   0, 140]],

        ...,

        [[  0,   0,   0, 214],
         [  0,   0,   0, 124],
         [  0,   0,   0, 124],
         ...,
         [  0,   0,   0, 124],
         [  0,   0,   0, 124],
         [  0,   0,   0,   0]],

        [[  0,   0,   0, 214],
         [  0,   0,   0, 124],
         [  0,   0,   0, 124],
         ...,
         [  0,   0,   0, 124],
         [  0,   0,   0,   0],
         

In [11]:
state, reward, done, info = env.step([5])

In [12]:
#plt.figure(figsize=(20,16))
#for idx in range(state.shape[3]):
 #   plt.subplot(1,4,idx+1)
 #   plt.imshow(state[0][:,:,idx])
#plt.show()

# 3. Train the RL Model

In [13]:
# AREA: Agent, Reward, Environment, Action
# PPO: Proximal Policy Optimization
# OS for file path management
import os
# PPO classifier
from stable_baselines3 import PPO
# Base Callback for saving models
from stable_baselines3.common.callbacks import BaseCallback

In [14]:
# Save our model every x number of steps
class TrainAndLoggingCallback(BaseCallback):
    
    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path
        
    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)
            
    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)
            
        return True

In [15]:
CHECKPOINT_DIR = './train/'
LOG_DIR = './logs/'

In [16]:
callback = TrainAndLoggingCallback(check_freq=1000000, save_path=CHECKPOINT_DIR)

In [17]:
# RL requires a policy network (such as the CNN that is the brain of the AI)
# CnnPolicy is fast at processing images, which fits our use case
model = PPO('CnnPolicy', env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.000001, n_steps=512)

Using cpu device
Wrapping the env in a VecTransposeImage.


In [ ]:
# Train the model
# Every single game gets a million frames
model.learn(total_timesteps=4000000, callback=callback)

Logging to ./logs/PPO_8


C:\Users\yacoo\AppData\Roaming\Python\Python310\site-packages\gym_super_mario_bros\smb_env.py:148: RuntimeWarning: overflow encountered in ubyte_scalars
  return (self.ram[0x86] - self.ram[0x071c]) % 256


----------------------------
| time/              |     |
|    fps             | 94  |
|    iterations      | 1   |
|    time_elapsed    | 5   |
|    total_timesteps | 512 |
----------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 17            |
|    iterations           | 2             |
|    time_elapsed         | 59            |
|    total_timesteps      | 1024          |
| train/                  |               |
|    approx_kl            | 3.6907964e-05 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.95         |
|    explained_variance   | 0.00358       |
|    learning_rate        | 1e-06         |
|    loss                 | 191           |
|    n_updates            | 10            |
|    policy_gradient_loss | -0.000509     |
|    value_loss           | 540           |
-------------------------------------------
-----

-------------------------------------------
| time/                   |               |
|    fps                  | 9             |
|    iterations           | 13            |
|    time_elapsed         | 721           |
|    total_timesteps      | 6656          |
| train/                  |               |
|    approx_kl            | 1.1670287e-05 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.95         |
|    explained_variance   | 0.00778       |
|    learning_rate        | 1e-06         |
|    loss                 | 0.0764        |
|    n_updates            | 120           |
|    policy_gradient_loss | -0.000158     |
|    value_loss           | 0.169         |
-------------------------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 9             |
|    iterations           | 14            |
|    time_elapsed         | 785 

------------------------------------------
| time/                   |              |
|    fps                  | 8            |
|    iterations           | 24           |
|    time_elapsed         | 1404         |
|    total_timesteps      | 12288        |
| train/                  |              |
|    approx_kl            | 1.982029e-05 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.94        |
|    explained_variance   | -0.000867    |
|    learning_rate        | 1e-06        |
|    loss                 | 0.491        |
|    n_updates            | 230          |
|    policy_gradient_loss | -0.000311    |
|    value_loss           | 1.43         |
------------------------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 8             |
|    iterations           | 25            |
|    time_elapsed         | 1470          |
|    t

...

------------------------------------------
| time/                   |              |
|    fps                  | 8            |
|    iterations           | 728          |
|    time_elapsed         | 42524        |
|    total_timesteps      | 372736       |
| train/                  |              |
|    approx_kl            | 0.0011995871 |
|    clip_fraction        | 0.00586      |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.402       |
|    explained_variance   | 0.0895       |
|    learning_rate        | 1e-06        |
|    loss                 | 571          |
|    n_updates            | 7270         |
|    policy_gradient_loss | -0.000834    |
|    value_loss           | 761          |
------------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 8            |
|    iterations           | 729          |
|    time_elapsed         | 42585        |
|    total_

-------------------------------------------
| time/                   |               |
|    fps                  | 8             |
|    iterations           | 739           |
|    time_elapsed         | 43174         |
|    total_timesteps      | 378368        |
| train/                  |               |
|    approx_kl            | 0.00044955336 |
|    clip_fraction        | 0.00605       |
|    clip_range           | 0.2           |
|    entropy_loss         | -0.886        |
|    explained_variance   | -0.303        |
|    learning_rate        | 1e-06         |
|    loss                 | 1.77          |
|    n_updates            | 7380          |
|    policy_gradient_loss | 0.00125       |
|    value_loss           | 25.7          |
-------------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 8            |
|    iterations           | 740          |
|    time_elapsed         | 43231   

------------------------------------------
| time/                   |              |
|    fps                  | 8            |
|    iterations           | 750          |
|    time_elapsed         | 43859        |
|    total_timesteps      | 384000       |
| train/                  |              |
|    approx_kl            | 0.0009872018 |
|    clip_fraction        | 0.00176      |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.516       |
|    explained_variance   | 0.664        |
|    learning_rate        | 1e-06        |
|    loss                 | 412          |
|    n_updates            | 7490         |
|    policy_gradient_loss | 0.00157      |
|    value_loss           | 527          |
------------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 8            |
|    iterations           | 751          |
|    time_elapsed         | 43929        |
|    total_

-------------------------------------------
| time/                   |               |
|    fps                  | 8             |
|    iterations           | 761           |
|    time_elapsed         | 44544         |
|    total_timesteps      | 389632        |
| train/                  |               |
|    approx_kl            | 5.3216238e-05 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -0.689        |
|    explained_variance   | 0.811         |
|    learning_rate        | 1e-06         |
|    loss                 | 104           |
|    n_updates            | 7600          |
|    policy_gradient_loss | 0.000141      |
|    value_loss           | 294           |
-------------------------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 8             |
|    iterations           | 762           |
|    time_elapsed         | 4460

-------------------------------------------
| time/                   |               |
|    fps                  | 8             |
|    iterations           | 772           |
|    time_elapsed         | 45197         |
|    total_timesteps      | 395264        |
| train/                  |               |
|    approx_kl            | 0.00066157384 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -0.594        |
|    explained_variance   | -0.0252       |
|    learning_rate        | 1e-06         |
|    loss                 | 115           |
|    n_updates            | 7710          |
|    policy_gradient_loss | -0.000979     |
|    value_loss           | 303           |
-------------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 8            |
|    iterations           | 773          |
|    time_elapsed         | 45255   

In [ ]:
# No callback: model.save('name_of_file')
# To view the logs on tensorboard
# Activate the env on this dir and go into log dir (go into most recent ppo model)
#tensorboard --logdir=.

# 4. Test the Model

In [ ]:
# Load the model
#model = PPO.load('./train/best_model_4000000')

In [ ]:
#state = env.reset()

#while True:
 #   action, _ = model.predict(state)
 #   state, reward, done, info = env.step(action)
 #   env.render()